In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Advanced Fork-Join Example 13
// Complex fork-join with 3 parallel branches
val model = Network("model")

In [ ]:
// Create network
val delay = Delay(model, "Delay1")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
val queue3 = Queue(model, "Queue3", SchedStrategy.PS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)

val jobclass1 = ClosedClass(model, "class1", 10, delay, 0)
val jobclass2 = ClosedClass(model, "class2", 10, delay, 0)

In [ ]:
// Service configurations
// Class 1
queue1.setService(jobclass1, Exp(1.5))
queue2.setService(jobclass1, Exp(1.1))
queue3.setService(jobclass1, Exp(2.5))
delay.setService(jobclass1, Exp(0.5))

// Class 2
queue1.setService(jobclass2, Exp(2.8))
queue2.setService(jobclass2, Exp(3.0))
queue3.setService(jobclass2, Exp(1.0))
delay.setService(jobclass2, Exp(0.8))

In [ ]:
// Set routing matrix with serial routing queue2->queue3
val P = model.initRoutingMatrix()

// Class 1 routing
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  // Serial: queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

// Class 2 routing
P.set(jobclass2, jobclass2, delay, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue2, queue3, 1.0)  // Serial: queue2 -> queue3
P.set(jobclass2, jobclass2, queue3, join, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, join, delay, 1.0)

model.link(P)

In [ ]:
// Solve with MVA (like MATLAB)
val solver = MVA(model)
try {
    val avgTable = solver.avgTable
    avgTable.print()
} catch (e: Exception) {
    println("Error: ${e.message}")
}

This three-branch fork-join model demonstrates:

1. **Asymmetric Branch Structure**:
   - **Branch 1**: Queue1 (direct path)
   - **Branch 2**: Queue2 → Queue3 (serial path)
   - Mixed parallel and serial processing within fork-join

2. **Multi-Class Competition**:
   - **Class 1**: 10 jobs with different service rates
   - **Class 2**: 10 jobs with different service rates
   - Both classes compete for all resources

3. **Heterogeneous Service Rates**:
   - **Class 1**: Queue1(1.5), Queue2(1.1), Queue3(2.5), Delay(0.5)
   - **Class 2**: Queue1(2.8), Queue2(3.0), Queue3(1.0), Delay(0.8)
   - Different performance characteristics per class

4. **Complex Synchronization**:
   - Join waits for both Queue1 and Queue3 completion
   - Queue3 receives load from Queue2 (serial dependency)
   - Bottleneck can shift between branches and classes

5. **Performance Analysis**:
   - **Class 1 bottleneck**: Likely Queue2 (rate 1.1) in serial path
   - **Class 2 bottleneck**: Likely Queue3 (rate 1.0) in serial path
   - **Overall**: Delay stations limit cycle rates

6. **Resource Utilization**:
   - Queue1: Light load (direct path)
   - Queue2: Moderate load (first in serial path)
   - Queue3: Heavy load (receives all Queue2 output)

The model represents:
- **Assembly lines**: Parallel processing with some serial sub-assemblies
- **Software systems**: Parallel tasks with dependencies
- **Manufacturing**: Mixed parallel and sequential operations
- **Workflow systems**: Complex business processes

Key insights:
- **Path asymmetry**: Serial path (Queue2→Queue3) likely dominates performance
- **Class differentiation**: Different service rates create class-specific bottlenecks
- **Load concentration**: Queue3 experiences compound load from both branches

**Performance Pattern**: The system performance is limited by the slowest component in the longest path, which is the Queue2→Queue3 serial section. Class 2 has a particularly slow Queue3 (rate 1.0), making it likely to be the overall system bottleneck.

**Design Insight**: This model shows how adding serial dependencies within parallel structures can create unexpected bottlenecks and load imbalances.